In [1]:
import os
import sys

# Add library path to Python path
sys.path.append(os.path.abspath('..'))

# Now import your modules
from library import libStopsPoints, libConnections, libHex
import zipfile
import os
import time
import pymongo as pym
import pandas as pd
import folium
import numpy as np
import requests
import numba

import math
import geopy
from shapely.geometry import Polygon, MultiPolygon, Point, mapping
from geopy.distance import geodesic,great_circle
from folium.plugins import FastMarkerCluster
from datetime import datetime
from geopy.distance import geodesic,great_circle
from pymongo import MongoClient

from pathlib import Path
import geopandas as gpd
import geojson
from IPython.core.display import display, HTML

from library import libStopsPoints, libConnections, libHex
 

# Url and paths  [**set it!**]

In [6]:
city = 'padam_territory_01'
scenario_name = 'PT_DRT_scenario_25' # basecase without drt

# file paths and addresses to be provided:
# population data
shpPath = '../Cities/Padam_terretory_01/shp files/opps.shp'
popCollectionName = "POP"
popField = "pop"

# Study area
# Study area defines the area in which the accessibility calaculation is executes
# study_area_shp_path = 'Paris/shp/studyarea.shp'


directoryGTFS = "../Cities/Padam_terretory_01/gtfs/PT_DRT/gtfs_PT_DRT_scenario_25/"

gtfs_prep_toggle = True
urlMongoDb = "mongodb://localhost:27017/"  # url of the mongodb database
urlMongoDbPop = "mongodb://localhost:27017/" # url of the mongodb database for population
urlServerOsrm = 'http://localhost:5000/' # url of the osrm server of the city

client = MongoClient(urlMongoDb)
scenario_name="PT_DRT_scenario_25" #drt
gtfsDB = client[str('pta-' + city + '-' +scenario_name)]
popDbName = str('pta-' + city + '-' +scenario_name)

# required parameters
# the date must be in the interval of validity of the gtfs files, check it in the "calendar.txt" and "calendar_dates.txt" files inside the gtfs zip files.
# the day for the computation of accessibility quantitites
days = ['20241201', '20241202']
day_names = ["sunday", "monday"]


# parameters of walking distance
timeWalk = 8 * 60  # seconds
velocityWalk = 1.31  # m/s ***4.71km/h***
distanceS = timeWalk * velocityWalk

# Parameters that define the resolution and extention of tesselletion and the maximum of the walking time
# grid step of the hexagonal tesselletion in kilometers
gridEdge = 1

# Set check4stops = False if cells / hexagones should be included that do not have stops within.
# Set check4stops = False for preprocessing prior to dynamic mode to gtfs convertion
# Set check4stops = True for citychrone accessibility analysis
check4stops = False

# Start of the computation

### add population data

In [3]:

shapefile = gpd.read_file(shpPath)
shapefile.to_file("../Cities/Padam_terretory_01/shp files/opps_for_padam_terretory_01.geojson", driver='GeoJSON')
with open("../Cities/Padam_terretory_01/shp files/opps_for_padam_terretory_01.geojson",encoding="utf8") as f:
    gj = geojson.load(f)
features = gj['features']
gtfsDB["POP"].drop()
gtfsDB["POP"].insert_many(features)


InsertManyResult([ObjectId('67765358bf5c038c2dafcc31'), ObjectId('67765358bf5c038c2dafcc32'), ObjectId('67765358bf5c038c2dafcc33'), ObjectId('67765358bf5c038c2dafcc34'), ObjectId('67765358bf5c038c2dafcc35'), ObjectId('67765358bf5c038c2dafcc36'), ObjectId('67765358bf5c038c2dafcc37'), ObjectId('67765358bf5c038c2dafcc38'), ObjectId('67765358bf5c038c2dafcc39'), ObjectId('67765358bf5c038c2dafcc3a'), ObjectId('67765358bf5c038c2dafcc3b'), ObjectId('67765358bf5c038c2dafcc3c'), ObjectId('67765358bf5c038c2dafcc3d'), ObjectId('67765358bf5c038c2dafcc3e'), ObjectId('67765358bf5c038c2dafcc3f'), ObjectId('67765358bf5c038c2dafcc40'), ObjectId('67765358bf5c038c2dafcc41'), ObjectId('67765358bf5c038c2dafcc42'), ObjectId('67765358bf5c038c2dafcc43'), ObjectId('67765358bf5c038c2dafcc44'), ObjectId('67765358bf5c038c2dafcc45'), ObjectId('67765358bf5c038c2dafcc46'), ObjectId('67765358bf5c038c2dafcc47'), ObjectId('67765358bf5c038c2dafcc48'), ObjectId('67765358bf5c038c2dafcc49'), ObjectId('67765358bf5c038c2dafcc

In [4]:
# load Gtfs data to mongodb database
listOfFile = ['stops.txt', 'routes.txt', 'trips.txt', 'calendar.txt', 'calendar_dates.txt',
              'stop_times.txt']  
libStopsPoints.loadGtfsFile(gtfsDB, directoryGTFS, city, listOfFile)

removing stops  of  padam_territory_01
removing routes  of  padam_territory_01
removing trips  of  padam_territory_01
removing calendar  of  padam_territory_01
removing calendar_dates  of  padam_territory_01
removing stop_times  of  padam_territory_01
gtfs_PT_DRT_scenario_25.zip
stops.txt -> (1108,1108)
routes.txt -> (588131,588131)
trips.txt -> (593857,593857)
calendar.txt -> (76,76)
calendar_dates.txt -> (85,85)
stop_times.txt -> (1198790,1198790)


## Fill the database with the connections

In [7]:

libConnections.readConnections(gtfsDB, city, directoryGTFS, days[0], day_names[0], overwrite=True)
libConnections.readConnections(gtfsDB, city, directoryGTFS, days[1], day_names[1], overwrite=False)


number of file in calendar+calendar_dates: 1
in stops: 1

Checking the number of services active in the date selected:
file: gtfs_PT_DRT_scenario_25.zip 	 total number of active service (in calendar.txt): 22
number of different service_id: 1


file: gtfs_PT_DRT_scenario_25.zip 	 total number of active service (in calendar_dates.txt): 23
number of different service_id: 1 total number of active services found: 23
number of trips 593000
 gtfs_PT_DRT_scenario_25.zip
inserting to DB....94688, err 0, err_start 0, err_start_after 0
tot connections 295909
number of file in calendar+calendar_dates: 1
in stops: 1

Checking the number of services active in the date selected:
file: gtfs_PT_DRT_scenario_25.zip 	 total number of active service (in calendar.txt): 35
number of different service_id: 1


file: gtfs_PT_DRT_scenario_25.zip 	 total number of active service (in calendar_dates.txt): 36
number of different service_id: 1 total number of active services found: 36
number of trips 593000
 gtfs_PT

## Remove stops with no connections and add to each stop the pos field

In [8]:
libStopsPoints.removingStopsNoConnections(gtfsDB, city)
libStopsPoints.setPosField(gtfsDB, city)

In [9]:
libConnections.updateConnectionsStopName(gtfsDB, city)

connections deleted 043 pEnd 599543, totC 599543


# Tassel with exagons

### List of all stops

In [10]:
stopsList = libStopsPoints.returnStopsList(gtfsDB, city)

tot stop 961  stop error : 0


## Compute the box that include all stops
The edge of such box are enlarged by distanceS.

In [11]:

display(HTML('<h1>All stops of the public transport present in the gtfs files</h1>'))
bbox = libStopsPoints.boundingBoxStops(stopsList)
libStopsPoints.mapStops(bbox, stopsList)

In [23]:
bbox

[1.8266460206188384, 48.85046916816086, 2.0514705793811614, 48.928343831839136]

## Tassel the box with exagons.

In [12]:
hexBin, pointBin = libHex.hexagonalGrid(bbox, gridEdge, gtfsDB['stops'], distanceS, city,check4stops)

In [13]:

libHex.insertPoints(pointBin, city, gtfsDB)


## Find the hex with walkingTime less than timeWalk from a stops

In [14]:

libHex.pointsServed(gtfsDB, stopsList, urlServerOsrm, distanceS, timeWalk, city)

In [15]:
print("Number of hexagons: {0}".format(gtfsDB['points'].count_documents({'served':True, 'city':city})))

Number of hexagons: 249


## Setting field "pos" for points for performance

In [16]:
from library import libHex 
libHex.settingHexsPos(gtfsDB, city)

## Setting Population of Hexagons

In [17]:
# from libHex import setHexsPop
import library
from library import libHex
if urlMongoDbPop != "" and popCollectionName != "":
    clientPop = pym.MongoClient(urlMongoDbPop)
    popDb = clientPop[popDbName]
    popCollection = popDb[popCollectionName]
    libHex.setHexsPop(gtfsDB, popCollection, popField, city)
else:
    print("Population NOT INSERTED!")

# res = gtfsDB['points'].update_many({'pop':{'$exists':False}}, {'$set':{'pop':0}})
# print("n° of matched hexagons with population Polygons: {0} \n \
# not matched: {1} (setted to zero)".format(gtfsDB['points'].count_documents({'pop':{'$exists':True}}),
#                                                                                      res.modified_count))

# Adding the walking time between stops and points

In [18]:
from library import libStopsPoints 
libStopsPoints.computeNeigh(gtfsDB, urlServerOsrm, distanceS, timeWalk,  city)

/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


# Compute quantities and observable

TimeList is the list of starting time for computing the isochrones

In [19]:
timeList = [7,10,12,17,20]

hStart = timeList[0]*3600

### List of connections

In [20]:
from library import libConnections 
# arrayCC = libConnections.makeArrayConnections(gtfsDB, hStart, city)
arrayCC_day1 = libConnections.makeArrayConnections(gtfsDB, hStart, city, day='20241201')
arrayCC_day2 = libConnections.makeArrayConnections(gtfsDB, hStart, city, day='20241202')

start making connections array
done recover all cc 295879
converted
Num of connection 295879
start making connections array
done recover all cc 302895
converted
Num of connection 302895


### List of list of the points and stops neighbors

In [21]:
from library import libStopsPoints 
arraySP = libStopsPoints.listPointsStopsN(gtfsDB, city)

## Compute accessibility quantities

In [ ]:
import imp
from library import libAccessibility_version2, shortest_path_algo
imp.reload(libAccessibility_version2)
from library.shortest_path_algo import computeAccessibilities
imp.reload(shortest_path_algo)

listAccessibility = ['accessibility30min']

computeIsochrone = False
if 'isochrones' in gtfsDB.list_collection_names():
    
    pass
for timeStart in timeList:
    timeStart *= 3600
    hEnd  = timeStart+6600 #consider only connection that start at timeStart and end before next timeStart in the list timeList
    print( 'Time Isochrone Start: {0}'.format(timeStart/3600,))
   
    computeAccessibilities(
    city, timeStart,hEnd, arrayCC_day1, arraySP, gtfsDB, 
    computeIsochrone, timeStart/3600 == timeList[0], 
    day='20241201', 
    max_travel_time=1800, #30min 
    listAccessibility=listAccessibility
)

    # For second day
    computeAccessibilities(
        city, timeStart,hEnd, arrayCC_day2, arraySP, gtfsDB,
        computeIsochrone, timeStart/3600 == timeList[0],  # Use first=True for each day
        day='20241202',
        max_travel_time=1800, #30min 
        listAccessibility=listAccessibility
    )